In [14]:
from objects import BallOnPlate

import random

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

import numpy
import multiprocessing

import os
import sys

import pickle
import time

import neat

from time import sleep
import math
import numpy as np

In [15]:
simulation_seconds = 10.

def eval_genome(genome):
    ballOnPlate = BallOnPlate(showGUI=False, randomInitial=False)

    cost = 0

    CONST_VALUE = 0.7
    intial_positions = [[CONST_VALUE, CONST_VALUE],
                        [-CONST_VALUE, -CONST_VALUE],
                        [-CONST_VALUE, CONST_VALUE],
                        [CONST_VALUE, -CONST_VALUE],
                        [0., 0.]]

    reference_positions = [[-CONST_VALUE, -CONST_VALUE],
                           [CONST_VALUE, CONST_VALUE],
                           [CONST_VALUE, -CONST_VALUE],
                           [-CONST_VALUE, CONST_VALUE],
                           [0., 0.]]

    for i in range(len(intial_positions)):

        envInput = [0, 0]
        result = 0
        dropDown = False

        ballOnPlate.intial_pos  = np.array(intial_positions[i])
        ref_point               = np.array(reference_positions[i])

        posOnPlate = ballOnPlate.reset()
        prevPosOnPlate = posOnPlate

        prev_err    = [0, 0]
        integr_err  = 0

        while ballOnPlate.time < simulation_seconds:

            # Get error
            err = ref_point - posOnPlate
            result -= (err[0] * err[0] + err[1] * err[1]) * (ballOnPlate.time + 1)

            ### PID controller
            prop    = float(genome[0])
            diff    = float(genome[1])
            integr  = float(genome[2])

            integr_err += err
            d_err = err - prev_err

            envInput[0] = prop * err[1] + diff * d_err[1] + integr_err[1] * integr
            envInput[0] = -envInput[0]
            envInput[1] = prop * err[0] + diff * d_err[0] + integr_err[0] * integr

            prev_err = err
            ### PID controller

            envInput = np.clip(envInput, -1, 1)

            prevPosOnPlate = posOnPlate

            posOnPlate, isEnd = ballOnPlate.step(envInput)
            if isEnd:
                # Bad penalty as fall
                dropDown = True
                break
            # sleep(ballOnPlate.dt)
        
        DROP_PENALTY_VALUE = 1e4 

        if dropDown:
            current_cost = (ballOnPlate.time * (DROP_PENALTY_VALUE/simulation_seconds) + result) - DROP_PENALTY_VALUE
        else:
            current_cost = result

        cost += current_cost / float(len(intial_positions))
        # cost = min(current_cost, cost)

    ballOnPlate.close()
    return cost,


creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator 
#                      define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
# toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("attr_float", random.random)

# Structure initializers
#                         define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_float, 3)

# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


#----------
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", eval_genome)

# register the crossover operator
toolbox.register("mate", tools.cxUniform, indpb=0.7)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutGaussian, indpb=0.1, mu=0, sigma=.02)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selTournament, tournsize=7)

In [ ]:
random.seed(time.time())

# Process Pool of 4 workers
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
toolbox.register("map", pool.map)

pop = toolbox.population(n=100)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

try:
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=10000, 
                        stats=stats, halloffame=hof)
except KeyboardInterrupt:
    print('Try-catch')

pool.close()

print("-- End of (successful) evolution --")

best_ind = tools.selBest(pop, 1)[0]
print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))


gen	nevals	avg    	std    	min     	max     
0  	100   	-8847.4	102.453	-8926.88	-8354.07
1  	64    	-8674.75	153.731	-8894.45	-8354.07
2  	58    	-8432.14	112.924	-8976.76	-8145.96
3  	69    	-8353.94	89.1552	-8884.82	-8079.96
4  	51    	-8276.14	145.067	-9178.33	-8063.38
5  	67    	-8149.86	158.377	-9164.51	-8063.38
6  	70    	-8104.61	210.805	-9200.96	-7198.14
7  	58    	-8025.06	194.126	-8476.66	-7198.14
8  	55    	-7851.09	447.452	-9241.69	-7118.13
9  	66    	-7315.8 	361.114	-9044.51	-7094.24
10 	68    	-7273.58	438.32 	-9239.94	-7056.2 
11 	68    	-7209.86	418.679	-9261.35	-7056.2 
12 	59    	-7118.27	284.71 	-9128.25	-7050.67
13 	58    	-7156.88	386.125	-9038.28	-7050.67
14 	68    	-7053.4 	7.25066	-7082   	-6998.11
15 	62    	-7086.53	211.134	-8484.65	-6997.29
16 	69    	-7083.28	311.772	-9027.21	-6960.72
17 	57    	-7028.03	234.821	-9165.99	-6960.72
18 	69    	-7056.94	339.322	-9122.85	-6947.38
19 	61    	-6973.05	150.785	-8453.43	-6725.45
20 	72    	-6981.78	268.72 	-9013.97